In [4]:
!pip install PyArabic

DEPRECATION: Loading egg at /home/ahmadsameh8/anaconda3/lib/python3.11/site-packages/translators-5.9.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 873.1 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [5]:
import pyarabic.araby as araby
import pandas as pd
import numpy as np
from datasets import DatasetDict, Dataset
import re

In [6]:
dataset = pd.read_csv("MergedDataset.csv")


# preprocessing of the dataset

In [7]:
class preprocessing:
    import re

    def clean_text(text):
        """Cleans text by removing English characters, mentions, URLs, and hashtags (except those with trailing characters), while preserving whitespace."""
        # cleaned_text = re.sub(r'[@A-Za-z]+|#[^\s]+|http\S+|@[^\s]+|\^[^ ]+', '', text)
        
        cleaned_text = re.sub(r'[@A-Za-z]+|#[^\s]+|http\S+|@[^\s]+|\^[^ ]+|\W+', ' ', text)
        
        
        return cleaned_text


    def normalize(text):
        '''
        Argument:
            string of words
        return:
            string of words but standardize the words
        '''

        text = text.strip()
        text = re.sub("[إأٱآا]", "ا", text)
        # text = re.sub("ى", "ي", text)
        # text = re.sub("ؤ", "ء", text)
        # text = re.sub("ئ", "ء", text)
        # text = re.sub("ة", "ه", text)
        noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
        text = re.sub(noise, '', text)
        text = re.sub(r'(.)\1+', r"\1\1", text) # Remove longation
        return araby.strip_tashkeel(text)
        
    
    def preprocess(sentence):
        '''
        Argument:
            string of words
        return:
            string of words after cleaning
        '''
        clean = preprocessing.clean_text(sentence)
        clean = preprocessing.normalize(clean)
        return clean


In [8]:
dataset["correct"] = dataset["correct"].apply(preprocessing.preprocess)
dataset["incorrect"] = dataset["incorrect"].apply(preprocessing.preprocess)


# Converting the dataset to a hugging face Object


In [ ]:
dataset_dict = {
    'correct': dataset["correct"].tolist(),
    'incorrect': dataset['incorrect'].tolist()
}
dataset_dict

In [37]:

hf_dataset = Dataset.from_dict(dataset_dict)
hf_dataset


Dataset({
    features: ['correct', 'incorrect'],
    num_rows: 22938
})

In [38]:
dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)
val_size = int(0.1 * dataset_size)
test_size = dataset_size - train_size - val_size

In [39]:
train_dataset = hf_dataset.select(range(train_size))
validation_dataset = hf_dataset.select(range(train_size, train_size + val_size))
test_dataset = hf_dataset.select(range(train_size + val_size, len(hf_dataset)))

In [40]:
print("Training set size:", train_size)
print("Validation set size:", val_size)
print("Testing set size:", test_size)


Training set size: 18350
Validation set size: 2293
Testing set size: 2295


In [41]:
dataset = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset,
    "test": test_dataset
})

for split_name, split_dataset in dataset.items():
    print(f"{split_name}: {split_dataset}")

# features = dataset["train"].features
# dataset = dataset.map(lambda example: {k: v for k, v in example.items() if k in features})

train: Dataset({
    features: ['correct', 'incorrect'],
    num_rows: 18350
})
validation: Dataset({
    features: ['correct', 'incorrect'],
    num_rows: 2293
})
test: Dataset({
    features: ['correct', 'incorrect'],
    num_rows: 2295
})


In [42]:

features = dataset["train"].features
dataset = dataset.map(lambda example: {k: v for k, v in example.items() if k in features})

# Pushing the dataset to hugging face

In [12]:
!pip install huggingface_hub


In [ ]:
# add hugging face token with access to write
# !huggingface-cli login

In [43]:
dataset.push_to_hub("QalbPreprocessedAndMergedwithoutPunct")

Pushing split train to the Hub.
Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:33<00:00, 33.33s/it]
Pushing split validation to the Hub.
Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:05<00:00,  5.97s/it]
Pushing split test to the Hub.
Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:07<00:00,  7.17s/it]
